In [1]:
from influxdb.commands import InfluxDBQueries, DataProcessor, LSTMModel
import warnings
import pandas as pd
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [2]:
df_train = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2024-05-01T01:00:00Z",
                                                stop="2025-01-01T01:00:00Z")
df_val = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2025-01-01T01:00:00Z",
                                              stop="2025-03-01T01:00:00Z")
df_test = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2025-03-01T01:00:00Z",
                                               stop="2025-04-01T01:00:00Z")

In [3]:
df_combined = pd.concat([df_train, df_test, df_val])

In [4]:
df_combined, scaler, columns, scaler_y = DataProcessor.add_features(df_combined)

In [5]:
df_combined.dropna(inplace=True)

In [6]:
columns, boruta = DataProcessor.select_features_boruta(df_combined, columns)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	31
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	31
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	31
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	31
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	31
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	31
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	31
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	7
Tentative: 	0
Rejected: 	24


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	7
Tentative: 	0
Rejected: 	24

------ Feature Selection Results ------

        Feature  Support  Ranking
       return_1     True        1
       return_5     True        1
 return_mean_24     True        1
  return_std_24     True        1
         atr_14     True        1
      range_cur     True        1
      range_pct     True        1
  actual_return    False        2
      macd_hist    False        3
        ma_diff    False      

In [7]:
print(columns, boruta)

['return_1', 'return_5', 'return_mean_24', 'return_std_24', 'atr_14', 'range_cur', 'range_pct'] BorutaPy(estimator=RandomForestRegressor(n_estimators=78, n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x30706A640),
         n_estimators='auto', perc=80,
         random_state=RandomState(MT19937) at 0x30706A640, verbose=2)


In [8]:
df_train = df_combined.loc[df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') < df_val.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]
df_val = df_combined.loc[(df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_val.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')) &
                         (df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') < df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ'))]
df_test = df_combined.loc[df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]

In [9]:
df_train = df_train.dropna()
df_test = df_test.dropna()
df_val = df_val.dropna()

In [10]:
X_train, y_train= DataProcessor.make_sequences(df_train, columns_to_select=columns, window_size=288, forecast_horizon=1)

In [11]:
X_val, y_val= DataProcessor.make_sequences(df_val, columns_to_select=columns, window_size=288, forecast_horizon=1)

In [14]:
lstm_forecaster = LSTMModel(input_shape=(288, len(columns)), output_steps=1)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 288, 32)        │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 144, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 144, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 144, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 144, 128)       │        82,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 144, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 144, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 1, 1)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 135,553 (529.50 KB)

 Trainable params: 135,105 (527.75 KB)

 Non-trainable params: 448 (1.75 KB)

In [15]:
lstm_forecaster.fit(X_train, y_train, X_val,y_val, epochs=200, batch_size=128, patience=10)

Training started with EarlyStopping (patience=10)...
Epoch 1/200
546/546 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step - directional_accuracy: 0.5007 - loss: 2.4961 - mse: 1.0843
Epoch 1: val_loss improved from inf to 2.12426, saving model to models/best_model.keras
546/546 ━━━━━━━━━━━━━━━━━━━━ 275s 499ms/step - directional_accuracy: 0.5007 - loss: 2.4958 - mse: 1.0841 - val_directional_accuracy: 0.5092 - val_loss: 2.1243 - val_mse: 1.0161 - learning_rate: 0.0010
Epoch 2/200
546/546 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step - directional_accuracy: 0.4982 - loss: 2.1471 - mse: 0.8700
Epoch 2: val_loss improved from 2.12426 to 2.07997, saving model to models/best_model.keras
546/546 ━━━━━━━━━━━━━━━━━━━━ 279s 512ms/step - directional_accuracy: 0.4982 - loss: 2.1471 - mse: 0.8700 - val_directional_accuracy: 0.5012 - val_loss: 2.0800 - val_mse: 1.0101 - learning_rate: 0.0010
Epoch 3/200
546/546 ━━━━━━━━━━━━━━━━━━━━ 0s 456ms/step - directional_accuracy: 0.5074 - loss: 2.0726 - mse: 0.8239
Epoch 3: val_loss im

In [16]:
X_test, y_test = DataProcessor.make_sequences(df_test, columns_to_select=columns, window_size=288, forecast_horizon=1)

In [17]:
pred = lstm_forecaster.predict(X_test)

270/270 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step
